In [ ]:
!pip install confluent_kafka

In [ ]:
!pip show confluent_kafka

In [ ]:
import json
import os
import pandas as pd
from dotenv import load_dotenv
from confluent_kafka import Producer

# .env 파일 로드
load_dotenv()

# Kafka 설정
conf = {
    "bootstrap.servers": os.getenv("BOOTSTRAP_SERVERS"),
    "security.protocol": "SASL_SSL",
    "sasl.mechanisms": "PLAIN",
    "sasl.username": os.getenv("SASL_USERNAME"),
    "sasl.password": os.getenv("SASL_PASSWORD"),
    "client.id": "json-serial-producer"
}
producer = Producer(conf)

# Kafka 토픽 설정
topic = "raw_topic"

# 메시지 전송 결과를 처리하는 콜백 함수
def delivery_report(err, msg):
    """
    Kafka로 메시지를 전송한 후 결과를 출력하는 콜백 함수
    :param err: 오류 정보 (있을 경우)
    :param msg: 전송된 메시지 정보
    """
    if err:
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered successfully! Key: {msg.key()}")

# 체크포인트를 읽는 함수 (이전 실행 위치 저장)
def read_checkpoint(checkpoint_file):
    """
    체크포인트 파일에서 마지막으로 처리한 라인의 인덱스를 읽어옴
    :param checkpoint_file: 체크포인트 파일 경로
    :return: 마지막으로 처리한 라인의 인덱스 (없으면 0 반환)
    """
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as file:
            return int(file.read().strip())
    return 0

# 체크포인트를 업데이트하는 함수
def write_checkpoint(checkpoint_file, index):
    """
    체크포인트 파일을 업데이트하여 마지막으로 처리한 라인의 인덱스를 저장
    :param checkpoint_file: 체크포인트 파일 경로
    :param index: 현재까지 처리한 마지막 라인 번호
    """
    with open(checkpoint_file, 'w') as file:
        file.write(str(index))
    print(f"Checkpoint updated to line: {index}")

# 날짜 데이터를 JSON 직렬화할 수 있도록 변환하는 함수
def handle_date(obj):
    """
    pandas.Timestamp 객체를 JSON 형식의 문자열로 변환
    :param obj: 변환할 객체
    :return: 문자열 형식의 날짜 (예: 'YYYY-MM-DD HH:MM:SS')
    """
    if isinstance(obj, pd.Timestamp):
        return obj.strftime('%Y-%m-%d %H:%M:%S')
    raise TypeError(f"Object of type {type(obj).__name__} is not JSON serializable")

# JSON 데이터를 한 줄씩 Kafka로 전송하는 함수
def stream_json_serially(file_path, checkpoint_file='/kaggle/working/checkpoint.txt'):
    """
    JSON 데이터를 한 줄씩 읽어 Kafka로 전송
    :param file_path: 입력 JSON 파일 경로
    :param checkpoint_file: 체크포인트 파일 경로 (이전 처리 위치 저장)
    """
    last_sent_index = read_checkpoint(checkpoint_file)

    with open(file_path, 'r') as file:
        for idx, line in enumerate(file):
            if idx < last_sent_index:
                continue

            try:
                record = json.loads(line)
                producer.produce(
                    topic,
                    key=str(record['review_id']),
                    value=json.dumps(record, default=handle_date).encode('utf-8'),
                    callback=delivery_report
                )

                producer.flush()
                write_checkpoint(checkpoint_file, idx + 1)

            except json.JSONDecodeError as e:
                print(f"Failed to decode JSON: {e}")

# 실행
if __name__ == "__main__":
    stream_json_serially('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')
